In [1]:
import tensorflow as tf
print("NUM GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2023-11-07 19:11:14.947536: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-07 19:11:14.948405: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-07 19:11:14.965727: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-07 19:11:14.966288: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-07 19:11:15.314559: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT

NUM GPUs Available:  0


2023-11-07 19:11:15.889783: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-07 19:11:15.899645: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [2]:
# data processing
# x_train is like past n days, y_train is future n days
# past n days will be 365 ? future n days is 7 ? for the week?
import pickle
import numpy as np
import pandas as pd
infile = open('data/aqi_by_county_data/cleaned_aqi_data.pkl', 'rb')
aqi_data = pickle.load(infile)
full_set = pd.DataFrame(aqi_data['AQI'])
training_set = full_set.iloc[:1676, :]
test_set = full_set.iloc[1676:, :]


In [3]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)

x_train = []
y_train = []
n_future = 4 # next 4 days temperature forecast
n_past = 30 # Past 30 days
for i in range(0,len(training_set_scaled)-n_past-n_future+1):
    x_train.append(training_set_scaled[i : i + n_past , 0])
    y_train.append(training_set_scaled[i + n_past : i + n_past + n_future , 0 ])
x_train , y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0] , x_train.shape[1], 1) )

In [ ]:
# data visualization

In [ ]:
# data standardizing

In [4]:
# model: recurrent neural network

# x_train and y_train should be numpy arrays

# https://medium.com/analytics-vidhya/weather-forecasting-with-recurrent-neural-networks-1eaa057d70c3

from keras.models import Sequential
from keras.layers import Bidirectional, LSTM,Dense ,Dropout


In [5]:

# set up regressor model
regressor = Sequential()
regressor.add(Bidirectional(LSTM(units=30, return_sequences=True, input_shape = (x_train.shape[1],1) ) ))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = n_future,activation='linear'))
regressor.compile(optimizer='adam', loss='mean_squared_error',metrics=['acc']) # adam optimizer for optimal efficiency
regressor.fit(x_train, y_train, epochs=500,batch_size=32 )


Epoch 1/500
52/52 [==============================] - 4s 15ms/step - loss: 0.0403 - acc: 0.2514
Epoch 2/500
52/52 [==============================] - 1s 15ms/step - loss: 0.0204 - acc: 0.2581
Epoch 3/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0184 - acc: 0.2593
Epoch 4/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0176 - acc: 0.2672
Epoch 5/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0171 - acc: 0.2422
Epoch 6/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0168 - acc: 0.2556
Epoch 7/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0157 - acc: 0.2465
Epoch 8/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0153 - acc: 0.2465
Epoch 9/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0153 - acc: 0.2392
Epoch 10/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0156 - acc: 0.2550
Epoch 11/500
52/52 [=============================

In [9]:
# read test dataset
testdataset = test_set
real_temperature = test_set
testing = sc.transform(testdataset)
testing = np.array(testing)
testing = np.reshape(testing,(testing.shape[1],testing.shape[0],1))

# predict model
predicted_temperature = regressor.predict(testing)
predicted_temperature = sc.inverse_transform(predicted_temperature)
predicted_temperature = np.reshape(predicted_temperature,(predicted_temperature.shape[1],predicted_temperature.shape[0]))

print(predicted_temperature)

1/1 [==============================] - 0s 24ms/step
[[23.847404]
 [21.79368 ]
 [19.517614]
 [21.689926]]
